In [1]:
import re
import os
import numpy as np
import pandas as pd
from pyecharts.charts import Bar
from pyecharts.charts import Line
from pyecharts.charts import Grid
from pyecharts.charts import Pie
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode

In [2]:
df1=pd.read_csv('足球比赛.csv',header=None,skiprows=[0])
df1.columns=['日期','主队队名','客队队名','主队分数','客队分数','赛事名','城市','国家','是否在中立场地进行']

In [3]:
#思路提纲
#1.各支球队分别在客场和主场的胜率（横向条形图）（已完成）
#2.最近五年（2014——2018），以主客场胜率前五的球队的主客场胜率的变化（横向条形图）
#3.以年份为自变量，主场坐镇的球队的胜率（主场优势的趋势，折线图）
#4.年份为自变量，进球总数，（折线图）
#5.球队实力如何看：主成分分析法（最为重要）
#6.每个国家举办的次数？那个国家举办的最多，而自己却没参加
#7.友谊赛道参赛次数场次前十的队伍
#8.

In [4]:
# df = df[df['A'] != 0]

In [5]:
#以主队为基准进行分析
df1['主队胜负情况']=df1['主队分数']-df1['客队分数']
df1['主队胜负情况'] = np.sign(df1['主队胜负情况'])
#1为胜出，0为平局，-1为负
df2 = pd.DataFrame(df1['主队队名'].value_counts() )
df11=df1[df1['主队胜负情况']==1]

In [6]:
 df2.columns=['总的主场场次']

In [7]:
df22=pd.DataFrame(df11['主队队名'].value_counts())
df22.columns=['胜出的场次']


In [8]:
df2['胜出场次']=df22['胜出的场次']
df3=df2.dropna(axis = 0)


In [9]:
df3['主场胜率']=df3['胜出场次']/df3['总的主场场次']
#由于总的主场场次过少可能说明该球队逐渐退出历史的舞台，所以只取总主场场次高于100场的
df3=df3[df3['总的主场场次']>=100]
df3

C:\Users\lenovo\AppData\Local\Temp\ipykernel_48716\4272577854.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['主场胜率']=df3['胜出场次']/df3['总的主场场次']


,总的主场场次,胜出场次,主场胜率
Brazil,552,395.0,0.715580
Argentina,535,354.0,0.661682
Germany,495,312.0,0.630303
Mexico,494,291.0,0.589069
England,483,298.0,0.616977
...,...,...,...
Faroe Islands,106,27.0,0.254717
Antigua and Barbuda,104,44.0,0.423077
Georgia,103,43.0,0.417476
Mauritius,102,35.0,0.343137


In [10]:
totalappear=df3['总的主场场次'].values.tolist()[::-1]
winappear=df3['胜出场次'].values.tolist()[::-1]
# f1 = lambda x:'%.2f%%' % (x*100)
# #df['主场胜率']=df3['主场胜率'].round(decimals=2)
# df3['主场胜率'] = pd.to_numeric(df3['主场胜率'])
# #df['主场胜率']=df3['主场胜率'].round(decimals=2)
# df3['主场胜率']=df3['主场胜率'].apply(f1)
#p_float = df['p_str'].str.strip("%").astype(float)/100;
#df3['主场胜率']=df3['主场胜率'].str.strip("%").astype(float)/100;
# winrate1=df3['主场胜率'].values.tolist()[::-1]
# winrate1=[round(i,2) for i in winrate1]
idx1=df3.index.tolist()[::-1]

In [11]:
df4=df3.sort_values(by="主场胜率" ,  ascending=False) 
df4

,总的主场场次,胜出场次,主场胜率
Brazil,552,395.0,0.715580
Spain,351,238.0,0.678063
Czech Republic,137,91.0,0.664234
Argentina,535,354.0,0.661682
Korea DPR,121,78.0,0.644628
...,...,...,...
Cyprus,195,52.0,0.266667
Faroe Islands,106,27.0,0.254717
Lesotho,121,26.0,0.214876
Malta,211,34.0,0.161137


In [12]:
winrate1=df4['主场胜率'].values.tolist()[::-1]
winrate1=[round(i,2) for i in winrate1]
idx2=df4.index.tolist()[::-1]

In [13]:
bar1 = (
    Bar()
    .add_xaxis(idx1)
    .add_yaxis("总的主场次数",totalappear, color='#EC407A')
    .add_yaxis("胜出的场次", winappear, color='#26A69A')
    .set_series_opts(
         label_opts=opts.LabelOpts(
            position='inside',
            vertical_align='middle',
         ),
        itemstyle_opts={
            'opacity': 0.9,
            'shadowBlur': 5,
            'shadowOffsetX': 5,
            'shadowOffsetY': 5,
            'shadowColor': 'rgba(125,0, 125, 0.5)',
            'barBorderRadius': [0, 15, 15, 0],
        }
      )
    .set_global_opts(
        tooltip_opts=opts.TooltipOpts(is_show=True, trigger='axis', axis_pointer_type='cross'),        
        datazoom_opts=opts.DataZoomOpts(orient="vertical",range_start=70,range_end=100),
        title_opts=opts.TitleOpts(
            title='主场胜场分析',
            
            pos_left='center',
            pos_top='1%',
            title_textstyle_opts=opts.TextStyleOpts(color='#ffb900', font_size=18),
        ),
        legend_opts=opts.LegendOpts(pos_left="center", pos_top='7%'),
        xaxis_opts=opts.AxisOpts(
            min_=0,
            max_=500,
            axislabel_opts=opts.LabelOpts(font_size=14, color='#c2ff00'),
            axisline_opts=opts.AxisLineOpts(
                is_show=False,
                linestyle_opts=opts.LineStyleOpts(width=2, color='#DB7093'))),
        yaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(font_size=14, color='#c2ff00'),
            axisline_opts=opts.AxisLineOpts(
                is_show=False,
                linestyle_opts=opts.LineStyleOpts(width=2, color='#DB7093')
            ),
        )
    )
    .reversal_axis()    
)

grid = Grid(init_opts=opts.InitOpts(width='1000px', height='1200px',bg_color='#0d0735'))
grid.add(bar1, opts.GridOpts(pos_top="10%",pos_left="15%", pos_right="15%"), is_control_axis_index=True)
grid.render_notebook()

In [14]:
bar1 = (
    Bar()
    .add_xaxis(idx2)
    .add_yaxis("总的主场次数",winrate1, color='#EC407A')
    .set_series_opts(
         label_opts=opts.LabelOpts(
            position='inside',
            vertical_align='middle',
         ),
        itemstyle_opts={
            'opacity': 0.9,
            'shadowBlur': 5,
            'shadowOffsetX': 5,
            'shadowOffsetY': 5,
            'shadowColor': 'rgba(125,0, 125, 0.5)',
            'barBorderRadius': [0, 15, 15, 0],
        }
      )
    .set_global_opts(
        tooltip_opts=opts.TooltipOpts(is_show=True, trigger='axis', axis_pointer_type='cross'),        
        datazoom_opts=opts.DataZoomOpts(orient="vertical",range_start=70,range_end=100),
        title_opts=opts.TitleOpts(
            title='主场胜率分析',
            
            pos_left='center',
            pos_top='1%',
            title_textstyle_opts=opts.TextStyleOpts(color='#ffb900', font_size=18),
        ),
        legend_opts=opts.LegendOpts(pos_left="center", pos_top='7%'),
        xaxis_opts=opts.AxisOpts(
            min_=0,
            max_=1,
            axislabel_opts=opts.LabelOpts(font_size=14, color='#c2ff00'),
            axisline_opts=opts.AxisLineOpts(
                is_show=False,
                linestyle_opts=opts.LineStyleOpts(width=2, color='#DB7093'))),
        yaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(font_size=14, color='#c2ff00'),
            axisline_opts=opts.AxisLineOpts(
                is_show=False,
                linestyle_opts=opts.LineStyleOpts(width=2, color='#DB7093')
            ),
        )
    )
    .reversal_axis()    
)

grid = Grid(init_opts=opts.InitOpts(width='1000px', height='1200px',bg_color='#0d0735'))
grid.add(bar1, opts.GridOpts(pos_top="10%",pos_left="15%", pos_right="15%"), is_control_axis_index=True)
grid.render_notebook()

# 对于客场球队的数据分析

In [15]:
df1['客队胜负情况']=df1['客队分数']-df1['主队分数']
df1['客队胜负情况'] = np.sign(df1['客队胜负情况'])
#1为胜出，0为平局，-1为负
df2 = pd.DataFrame(df1['客队队名'].value_counts() )
df11=df1[df1['客队胜负情况']==1]
df11

,日期,主队队名,客队队名,主队分数,客队分数,赛事名,城市,国家,是否在中立场地进行,主队胜负情况,客队胜负情况
6,1877-03-03,England,Scotland,1,3,Friendly,London,England,False,-1,1
7,1877-03-05,Wales,Scotland,0,2,Friendly,Wrexham,Wales,False,-1,1
12,1879-04-07,Wales,Scotland,0,3,Friendly,Wrexham,Wales,False,-1,1
14,1880-03-15,Wales,England,2,3,Friendly,Wrexham,Wales,False,-1,1
16,1881-02-26,England,Wales,0,1,Friendly,Blackburn,England,False,-1,1
...,...,...,...,...,...,...,...,...,...,...,...
39653,2018-06-28,Panama,Tunisia,1,2,FIFA World Cup,Saransk,Russia,True,-1,1
39654,2018-06-28,England,Belgium,0,1,FIFA World Cup,Kaliningrad,Russia,True,-1,1
39664,2018-07-06,Uruguay,France,0,2,FIFA World Cup,Nizhny Novgorod,Russia,True,-1,1
39665,2018-07-06,Brazil,Belgium,1,2,FIFA World Cup,Kazan,Russia,True,-1,1


In [16]:
df2.columns=['总的客场场次']

In [17]:
df22=pd.DataFrame(df11['客队队名'].value_counts())
df22.columns=['胜出的场次']
df22

,胜出的场次
England,259
Germany,234
Brazil,215
Sweden,212
Uruguay,188
...,...
Anguilla,1
Northern Mariana Islands,1
Tuvalu,1
Saarland,1


In [18]:
df2['胜出场次']=df22['胜出的场次']
df3=df2.dropna(axis = 0)

In [19]:
df3['客场胜率']=df3['胜出场次']/df3['总的客场场次']
#由于总的主场场次过少可能说明该球队逐渐退出历史的舞台，所以只取总主场场次高于100场的
df3=df3[df3['总的客场场次']>=100]

C:\Users\lenovo\AppData\Local\Temp\ipykernel_48716\2679111673.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['客场胜率']=df3['胜出场次']/df3['总的客场场次']


In [20]:
totalappear=df3['总的客场场次'].values.tolist()[::-1]
winappear=df3['胜出场次'].values.tolist()[::-1]
idx1=df3.index.tolist()[::-1]

In [21]:
df4=df3.sort_values(by="客场胜率" ,  ascending=False) 
df4

,总的客场场次,胜出场次,客场胜率
Germany,438,234.0,0.534247
Brazil,405,215.0,0.530864
England,499,259.0,0.519038
Spain,323,153.0,0.473684
Croatia,157,73.0,0.464968
...,...,...,...
Mauritania,112,10.0,0.089286
Cyprus,151,13.0,0.086093
Bolivia,210,17.0,0.080952
Malta,169,12.0,0.071006


In [22]:
idx2=df4.index.tolist()[::-1]

In [23]:
winrate1=df4['客场胜率'].values.tolist()[::-1]
winrate1=[round(i,2) for i in winrate1]
idx2=df4.index.tolist()[::-1]

In [24]:
bar1 = (
    Bar()
    .add_xaxis(idx1)
    .add_yaxis("总的客场次数",totalappear, color='#EC407A')
    .add_yaxis("胜出的场次", winappear, color='#26A69A')
    .set_series_opts(
         label_opts=opts.LabelOpts(
            position='inside',
            vertical_align='middle',
         ),
        itemstyle_opts={
            'opacity': 0.9,
            'shadowBlur': 5,
            'shadowOffsetX': 5,
            'shadowOffsetY': 5,
            'shadowColor': 'rgba(125,0, 125, 0.5)',
            'barBorderRadius': [0, 15, 15, 0],
        }
      )
    .set_global_opts(
        tooltip_opts=opts.TooltipOpts(is_show=True, trigger='axis', axis_pointer_type='cross'),        
        datazoom_opts=opts.DataZoomOpts(orient="vertical",range_start=70,range_end=100),
        title_opts=opts.TitleOpts(
            title='客场胜场分析',
            
            pos_left='center',
            pos_top='1%',
            title_textstyle_opts=opts.TextStyleOpts(color='#ffb900', font_size=18),
        ),
        legend_opts=opts.LegendOpts(pos_left="center", pos_top='7%'),
        xaxis_opts=opts.AxisOpts(
            min_=0,
            max_=500,
            axislabel_opts=opts.LabelOpts(font_size=14, color='#c2ff00'),
            axisline_opts=opts.AxisLineOpts(
                is_show=False,
                linestyle_opts=opts.LineStyleOpts(width=2, color='#DB7093'))),
        yaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(font_size=14, color='#c2ff00'),
            axisline_opts=opts.AxisLineOpts(
                is_show=False,
                linestyle_opts=opts.LineStyleOpts(width=2, color='#DB7093')
            ),
        )
    )
    .reversal_axis()    
)

grid = Grid(init_opts=opts.InitOpts(width='1000px', height='1200px',bg_color='#0d0735'))
grid.add(bar1, opts.GridOpts(pos_top="10%",pos_left="15%", pos_right="15%"), is_control_axis_index=True)
grid.render_notebook()

In [25]:
bar1 = (
    Bar()
    .add_xaxis(idx2)
    .add_yaxis("总的客场次数",winrate1, color='#EC407A')
    .set_series_opts(
         label_opts=opts.LabelOpts(
            position='inside',
            vertical_align='middle',
         ),
        itemstyle_opts={
            'opacity': 0.9,
            'shadowBlur': 5,
            'shadowOffsetX': 5,
            'shadowOffsetY': 5,
            'shadowColor': 'rgba(125,0, 125, 0.5)',
            'barBorderRadius': [0, 15, 15, 0],
        }
      )
    .set_global_opts(
        tooltip_opts=opts.TooltipOpts(is_show=True, trigger='axis', axis_pointer_type='cross'),        
        datazoom_opts=opts.DataZoomOpts(orient="vertical",range_start=70,range_end=100),
        title_opts=opts.TitleOpts(
            title='客场胜率分析',
            
            pos_left='center',
            pos_top='1%',
            title_textstyle_opts=opts.TextStyleOpts(color='#ffb900', font_size=18),
        ),
        legend_opts=opts.LegendOpts(pos_left="center", pos_top='7%'),
        xaxis_opts=opts.AxisOpts(
            min_=0,
            max_=1,
            axislabel_opts=opts.LabelOpts(font_size=14, color='#c2ff00'),
            axisline_opts=opts.AxisLineOpts(
                is_show=False,
                linestyle_opts=opts.LineStyleOpts(width=2, color='#DB7093'))),
        yaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(font_size=14, color='#c2ff00'),
            axisline_opts=opts.AxisLineOpts(
                is_show=False,
                linestyle_opts=opts.LineStyleOpts(width=2, color='#DB7093')
            ),
        )
    )
    .reversal_axis()    
)

grid = Grid(init_opts=opts.InitOpts(width='1000px', height='1200px',bg_color='#0d0735'))
grid.add(bar1, opts.GridOpts(pos_top="10%",pos_left="15%", pos_right="15%"), is_control_axis_index=True)
grid.render_notebook()

# 球队主客场胜率随年份变化的关系(这个暂时终止)

In [26]:
#筛选出特定时间段的数据

In [27]:
listdate=df1['日期'].values.tolist()
listdate=[i.split("-") for i in listdate]
listdate=[i[0] for i in listdate]
listdate=[int(i) for i in listdate]

In [28]:
df1['日期']=listdate

In [29]:
df1

,日期,主队队名,客队队名,主队分数,客队分数,赛事名,城市,国家,是否在中立场地进行,主队胜负情况,客队胜负情况
0,1872,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,0,0
1,1873,England,Scotland,4,2,Friendly,London,England,False,1,-1
2,1874,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,1,-1
3,1875,England,Scotland,2,2,Friendly,London,England,False,0,0
4,1876,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,1,-1
...,...,...,...,...,...,...,...,...,...,...,...
39664,2018,Uruguay,France,0,2,FIFA World Cup,Nizhny Novgorod,Russia,True,-1,1
39665,2018,Brazil,Belgium,1,2,FIFA World Cup,Kazan,Russia,True,-1,1
39666,2018,Sweden,England,0,2,FIFA World Cup,Samara,Russia,True,-1,1
39667,2018,Russia,Croatia,2,2,FIFA World Cup,Sochi,Russia,False,0,0


# 关于哪支球队主宰了不同的足球时代

In [30]:
#以比赛的规模来确定足球时代？value_counts()

In [31]:
dffootballteam=pd.DataFrame(df1['日期'].value_counts())
dffootballteam.sort_index(axis=0,ascending=True)
dffootballteam.rename(columns={'日期':'举办总场数'},inplace=True) 
dffootballteam['日期']=dffootballteam.index
dffootballteam.sort_values(by='日期', inplace=True)
dffootballteam

,举办总场数,日期
1872,1,1872
1873,1,1873
1874,1,1874
1875,1,1875
1876,2,1876
...,...,...
2014,857,2014
2015,1001,2015
2016,918,2016
2017,908,2017


In [32]:
import pyecharts.options as opts
from pyecharts.charts import Line
x_data=list(dffootballteam['日期'])
y_data = list(dffootballteam['举办总场数'])
x_data=[str(i) for i in x_data]
y_data=[str(i) for i in y_data]
line=(
    Line()
    .add_xaxis(xaxis_data=x_data)
    .add_yaxis(
        series_name="场数数量",
        y_axis=y_data,
        is_smooth=True,
        label_opts=opts.LabelOpts(is_show=True),
        linestyle_opts=opts.LineStyleOpts(width=3),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="举办比赛场数", subtitle="比赛规模",title_textstyle_opts=opts.TextStyleOpts(color='#ffb900', font_size=18)),
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
        xaxis_opts=opts.AxisOpts(boundary_gap=False),
        yaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(formatter="{value} 场"),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        visualmap_opts=opts.VisualMapOpts(
            is_piecewise=False,
            dimension=0,
            pos_right=0,
            pos_bottom=200
        ),
    )
    .set_series_opts(
        markarea_opts=opts.MarkAreaOpts(
            data=[
                opts.MarkAreaItem(name="高峰", x=("2000", "2016")),
            ]
        )
    )
)
line.render_notebook()

# 根据上图大概将足球的历史发展分为：萌芽期（1872----1944），飞速发展期（1944----2000），巅峰波折期（2000---2016）

In [33]:
#以高峰期为例

In [34]:
yearsstr=[i for i in range(2000,2017)]
dfspeeddvp=df1[df1['日期'].isin(yearsstr)]

In [35]:
#对高峰期的强队主客场分析
dfpeak = pd.DataFrame(dfspeeddvp['主队队名'].value_counts())

dfpeak.columns=['总的主场场次']

#筛选出胜利场次
dfspeeddvpwin=dfspeeddvp[dfspeeddvp['主队胜负情况']==1]


dfspeeddvpwin=pd.DataFrame(dfspeeddvpwin['主队队名'].value_counts())

dfspeeddvpwin.columns=['胜出场次']

dfpeak['胜出场次']=dfspeeddvpwin['胜出场次']
dfpeak
dfpeak=dfpeak.dropna(axis=0)
dfpeak['主场胜率']=dfpeak['胜出场次']/dfpeak['总的主场场次']
dfpeak=dfpeak[dfpeak['总的主场场次']>=100]

#转列表传参
totalappear_peak=dfpeak['总的主场场次'].values.tolist()[::-1]
winappear_peak=dfpeak['胜出场次'].values.tolist()[::-1]

dfpeak=dfpeak.sort_values(by="主场胜率" ,  ascending=False) 
peakwinrate=dfpeak['主场胜率'].values.tolist()[::-1]
peakwinrate=[round(i,2) for i in peakwinrate]
idxpeak=dfpeak.index.tolist()[::-1]

In [36]:
bar1 = (
    Bar()
    .add_xaxis(idxpeak)
    .add_yaxis("总的主场次数",totalappear_peak, color='#EC407A')
    .add_yaxis("胜出的场次", winappear_peak, color='#26A69A')
    .set_series_opts(
         label_opts=opts.LabelOpts(
            position='inside',
            vertical_align='middle',
         ),
        itemstyle_opts={
            'opacity': 0.9,
            'shadowBlur': 5,
            'shadowOffsetX': 5,
            'shadowOffsetY': 5,
            'shadowColor': 'rgba(125,0, 125, 0.5)',
            'barBorderRadius': [0, 15, 15, 0],
        }
      )
    .set_global_opts(
        tooltip_opts=opts.TooltipOpts(is_show=True, trigger='axis', axis_pointer_type='cross'),        
        datazoom_opts=opts.DataZoomOpts(orient="vertical",range_start=70,range_end=100),
        title_opts=opts.TitleOpts(
            title='巅峰期主场胜场分析',
            
            pos_left='center',
            pos_top='1%',
            title_textstyle_opts=opts.TextStyleOpts(color='#ffb900', font_size=18),
        ),
        legend_opts=opts.LegendOpts(pos_left="center", pos_top='7%'),
        xaxis_opts=opts.AxisOpts(
            min_=0,
            max_=200,
            axislabel_opts=opts.LabelOpts(font_size=14, color='#c2ff00'),
            axisline_opts=opts.AxisLineOpts(
                is_show=False,
                linestyle_opts=opts.LineStyleOpts(width=2, color='#DB7093'))),
        yaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(font_size=14, color='#c2ff00'),
            axisline_opts=opts.AxisLineOpts(
                is_show=False,
                linestyle_opts=opts.LineStyleOpts(width=2, color='#DB7093')
            ),
        )
    )
    .reversal_axis()    
)

grid = Grid(init_opts=opts.InitOpts(width='1000px', height='1200px',bg_color='#0d0735'))
grid.add(bar1, opts.GridOpts(pos_top="10%",pos_left="15%", pos_right="15%"), is_control_axis_index=True)
grid.render_notebook()

In [37]:
bar1 = (
    Bar()
    .add_xaxis(idxpeak)
    .add_yaxis("总的主场次数",peakwinrate, color='#EC407A')
    .set_series_opts(
         label_opts=opts.LabelOpts(
            position='inside',
            vertical_align='middle',
         ),
        itemstyle_opts={
            'opacity': 0.9,
            'shadowBlur': 5,
            'shadowOffsetX': 5,
            'shadowOffsetY': 5,
            'shadowColor': 'rgba(125,0, 125, 0.5)',
            'barBorderRadius': [0, 15, 15, 0],
        }
      )
    .set_global_opts(
        tooltip_opts=opts.TooltipOpts(is_show=True, trigger='axis', axis_pointer_type='cross'),        
        datazoom_opts=opts.DataZoomOpts(orient="vertical",range_start=70,range_end=100),
        title_opts=opts.TitleOpts(
            title='巅峰期主场胜率分析',
            
            pos_left='center',
            pos_top='1%',
            title_textstyle_opts=opts.TextStyleOpts(color='#ffb900', font_size=18),
        ),
        legend_opts=opts.LegendOpts(pos_left="center", pos_top='7%'),
        xaxis_opts=opts.AxisOpts(
            min_=0,
            max_=1,
            axislabel_opts=opts.LabelOpts(font_size=14, color='#c2ff00'),
            axisline_opts=opts.AxisLineOpts(
                is_show=False,
                linestyle_opts=opts.LineStyleOpts(width=2, color='#DB7093'))),
        yaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(font_size=14, color='#c2ff00'),
            axisline_opts=opts.AxisLineOpts(
                is_show=False,
                linestyle_opts=opts.LineStyleOpts(width=2, color='#DB7093')
            ),
        )
    )
    .reversal_axis()    
)

grid = Grid(init_opts=opts.InitOpts(width='1000px', height='1200px',bg_color='#0d0735'))
grid.add(bar1, opts.GridOpts(pos_top="10%",pos_left="15%", pos_right="15%"), is_control_axis_index=True)
grid.render_notebook()

# 在2000---2016年这一段足球黄金时期，西班牙，埃及，巴西属于前三的霸主地位，再从之前整个足球的历史来看，巴西为当之无愧的霸主

# 总进球数的历史趋势

In [38]:
df1['总进球数']=df1['主队分数']+df1['客队分数']
df_init=df1.groupby(['日期']).sum()
df_init['日期']=df_init.index
df_init

,主队分数,客队分数,是否在中立场地进行,主队胜负情况,客队胜负情况,总进球数,日期
日期,,,,,,,
1872,0,0,0,0,0,0,1872
1873,4,2,0,1,-1,6,1873
1874,2,1,0,1,-1,3,1874
1875,2,2,0,0,0,4,1875
1876,7,0,0,2,-2,7,1876
...,...,...,...,...,...,...,...
2014,1258,986,291,131,-131,2244,2014
2015,1568,1108,280,170,-170,2676,2015
2016,1506,989,247,194,-194,2495,2016


In [39]:
import pyecharts.options as opts
from pyecharts.charts import Line
x_data=list(df_init['日期'])
y_data = list(df_init['总进球数'])
x_data=[str(i) for i in x_data]
y_data=[str(i) for i in y_data]
line=(
    Line()
    .add_xaxis(xaxis_data=x_data)
    .add_yaxis(
        series_name="总的进球数",
        y_axis=y_data,
        is_smooth=True,
        label_opts=opts.LabelOpts(is_show=True),
        linestyle_opts=opts.LineStyleOpts(width=3),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="总共进球数", subtitle="比赛规模",title_textstyle_opts=opts.TextStyleOpts(color='#ffb900', font_size=18)),
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
        xaxis_opts=opts.AxisOpts(boundary_gap=False),
        yaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(formatter="{value} 球"),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        visualmap_opts=opts.VisualMapOpts(
            is_piecewise=False,
            dimension=0,
            pos_right=0,
            pos_bottom=200
        ),
    )
    .set_series_opts(
        markarea_opts=opts.MarkAreaOpts(
            data=[
                opts.MarkAreaItem(name="高峰", x=("2001", "2016")),
                opts.MarkAreaItem(name="波折期(可能与地缘政治因素有关)", x=("1938", "1947")),
            ]
        )
    )
)
line.render_notebook()

# 举办国家数量的变化

In [40]:
df1

,日期,主队队名,客队队名,主队分数,客队分数,赛事名,城市,国家,是否在中立场地进行,主队胜负情况,客队胜负情况,总进球数
0,1872,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,0,0,0
1,1873,England,Scotland,4,2,Friendly,London,England,False,1,-1,6
2,1874,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,1,-1,3
3,1875,England,Scotland,2,2,Friendly,London,England,False,0,0,4
4,1876,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,1,-1,3
...,...,...,...,...,...,...,...,...,...,...,...,...
39664,2018,Uruguay,France,0,2,FIFA World Cup,Nizhny Novgorod,Russia,True,-1,1,2
39665,2018,Brazil,Belgium,1,2,FIFA World Cup,Kazan,Russia,True,-1,1,3
39666,2018,Sweden,England,0,2,FIFA World Cup,Samara,Russia,True,-1,1,2
39667,2018,Russia,Croatia,2,2,FIFA World Cup,Sochi,Russia,False,0,0,4


In [41]:
# pd.set_option('display.max_rows', None)
# df2=df1.groupby(['日期','主队队名']).sum()

In [42]:
df2=pd.DataFrame(df1['国家'].groupby(df1['日期']).nunique())

In [43]:
#下一步对df2中的['参赛队伍进行去重']

In [44]:

df2['日期1']=df2.index
df2.sort_values(by='日期1', inplace=True)
df2

,国家,日期1
日期,,
1872,1,1872
1873,1,1873
1874,1,1874
1875,1,1875
1876,1,1876
...,...,...
2014,159,2014
2015,185,2015
2016,188,2016


In [45]:
import pyecharts.options as opts
from pyecharts.charts import Line
x_data=list(df2['日期1'])
y_data = list(df2['国家'])
x_data=[str(i) for i in x_data]
y_data=[str(i) for i in y_data]
line=(
    Line()
    .add_xaxis(xaxis_data=x_data)
    .add_yaxis(
        series_name="举办的国家数量",
        y_axis=y_data,
        is_smooth=True,
        label_opts=opts.LabelOpts(is_show=True),
        linestyle_opts=opts.LineStyleOpts(width=3),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="举办国家的数量统计", subtitle="比赛规模",title_textstyle_opts=opts.TextStyleOpts(color='#ffb900', font_size=18)),
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
        xaxis_opts=opts.AxisOpts(boundary_gap=False),
        yaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(formatter="{value} 个"),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        visualmap_opts=opts.VisualMapOpts(
            is_piecewise=False,
            dimension=0,
            pos_right=0,
            pos_bottom=200
        ),
    )
    .set_series_opts(
        markarea_opts=opts.MarkAreaOpts(
            data=[
                opts.MarkAreaItem(name="高峰", x=("2000", "2016")),
            ]
        )
    )
)
line.render_notebook()

# 从以上几幅折线图的走势来看，都极大反映了足球时代的低谷期受到了第二次世界大战的影响（历史政治问题）

# 关于哪些国家喜欢相互比赛

In [46]:
# dfchange1=df1
# dfchange1.columns=['日期','客队队名','主队队名','主队分数','客队分数','赛事名','城市','国家','是否在中立场地进行','主队胜负情况','客队胜负情况','总进球数']
df1['交手双方']=df1['主队队名']+'.VS.'+df1['客队队名']

In [47]:
dftogether1=pd.DataFrame(df1['交手双方'].value_counts())
dftogether1

,交手双方
Argentina.VS.Uruguay,95
Uruguay.VS.Argentina,80
Austria.VS.Hungary,68
Hungary.VS.Austria,65
Kenya.VS.Uganda,64
...,...
Armenia.VS.Norway,1
Slovakia.VS.Moldova,1
San Marino.VS.Croatia,1
Italy.VS.Morocco,1


In [48]:
str='.'
listtogether=dftogether1.index.tolist()
for i in  range(len(listtogether)):
    list1=listtogether[i].split('.')
    list2=list1[::-1]
    a= str.join(list2)
    if  a in  listtogether:
        listtogether[i]=a
dftogether1.index=listtogether

In [49]:
dftogether1['交战双方']=dftogether1.index.tolist()

In [50]:
dftogether1

,交手双方,交战双方
Uruguay.VS.Argentina,95,Uruguay.VS.Argentina
Uruguay.VS.Argentina,80,Uruguay.VS.Argentina
Hungary.VS.Austria,68,Hungary.VS.Austria
Hungary.VS.Austria,65,Hungary.VS.Austria
Uganda.VS.Kenya,64,Uganda.VS.Kenya
...,...,...
Armenia.VS.Norway,1,Armenia.VS.Norway
Slovakia.VS.Moldova,1,Slovakia.VS.Moldova
San Marino.VS.Croatia,1,San Marino.VS.Croatia
Italy.VS.Morocco,1,Italy.VS.Morocco


In [51]:
dftogether1.index=[i for i in range(0,10502)]

In [52]:
dftogether2=pd.DataFrame(dftogether1.groupby(['交战双方'])['交手双方'].sum())
pd.set_option('display.max_rows', None)
dftogether3=pd.DataFrame(dftogether2.sort_values(by='交手双方',ascending=False))
dftogether3

,交手双方
交战双方,
Uruguay.VS.Argentina,175
Hungary.VS.Austria,133
Belgium.VS.Netherlands,124
England.VS.Scotland,112
Scotland.VS.Wales,106
Uganda.VS.Kenya,106
Denmark.VS.Sweden,106
Sweden.VS.Norway,105
Brazil.VS.Argentina,102


In [53]:
x_aix=dftogether3.index.tolist()[::-1]
y_aiy=dftogether3['交手双方'].tolist()[::-1]

In [54]:
#将交手频次最多的队伍两两处理完毕

In [55]:
bar1 = (
    Bar()
    .add_xaxis(x_aix)
    .add_yaxis("交战的双方",y_aiy, color='#EC407A')
    .set_series_opts(
         label_opts=opts.LabelOpts(
            position='inside',
            vertical_align='middle',
         ),
        itemstyle_opts={
            'opacity': 0.9,
            'shadowBlur': 5,
            'shadowOffsetX': 5,
            'shadowOffsetY': 5,
            'shadowColor': 'rgba(125,0, 125, 0.5)',
            'barBorderRadius': [0, 15, 15, 0],
        }
      )
    .set_global_opts(
        tooltip_opts=opts.TooltipOpts(is_show=True, trigger='axis', axis_pointer_type='cross'),        
        datazoom_opts=opts.DataZoomOpts(orient="vertical",range_start=70,range_end=100),
        title_opts=opts.TitleOpts(
            title='交战次数最多的队伍分析',
            
            pos_left='center',
            pos_top='1%',
            title_textstyle_opts=opts.TextStyleOpts(color='#ffb900', font_size=18),
        ),
        legend_opts=opts.LegendOpts(pos_left="center", pos_top='7%'),
        xaxis_opts=opts.AxisOpts(
            min_=0,
            max_=200,
            axislabel_opts=opts.LabelOpts(font_size=14, color='#c2ff00'),
            axisline_opts=opts.AxisLineOpts(
                is_show=False,
                linestyle_opts=opts.LineStyleOpts(width=2, color='#DB7093'))),
        yaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(font_size=14, color='#c2ff00'),
            axisline_opts=opts.AxisLineOpts(
                is_show=False,
                linestyle_opts=opts.LineStyleOpts(width=2, color='#DB7093')
            ),
        )
    )
    .reversal_axis()    
)

grid = Grid(init_opts=opts.InitOpts(width='1000px', height='1200px',bg_color='#0d0735'))
grid.add(bar1, opts.GridOpts(pos_top="10%",pos_left="15%", pos_right="15%"), is_control_axis_index=True)
grid.render_notebook()

# 关于哪些球队最热衷于参加的比赛类型

# 历史中各种比赛类型的比例

In [56]:
df_competitiontype=pd.DataFrame(df1.groupby(['赛事名'])['日期'].count())

In [57]:
df_competitiontype1=df_competitiontype.sort_values(by='日期',ascending=False)

In [58]:
df_competitiontype1

,日期
赛事名,
Friendly,16475
FIFA World Cup qualification,7100
UEFA Euro qualification,2332
African Cup of Nations qualification,1558
FIFA World Cup,897
Copa América,787
AFC Asian Cup qualification,724
African Cup of Nations,638
CECAFA Cup,620


In [59]:
compet_name=df_competitiontype1.index.tolist()
compet_num=df_competitiontype1['日期'].tolist()
compet_num=[round(i/sum(compet_num)*100,2 )for i in compet_num]

In [62]:
colors = ["#00BCD4","#ea1d5d", "#ffb900", "#4FC3F7"]
color_series = ['#C9DA36','#37B44E','#1E91CA','#6A368B','#D5225B','#CF7B25']
P = (
    Pie(init_opts=opts.InitOpts(width='1500px', height='1000px',bg_color='#0d0735'))
    .add(
        "",
        [list(z) for z in zip(compet_name, compet_num)],
        radius=["25%", "70%"],
        center=["50%", "50%"],
        rosetype="radius",
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_colors(color_series)
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}%",font_size=16))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="各个类型赛事的比例", pos_top="40%", pos_right="center",
                                 title_textstyle_opts=opts.TextStyleOpts(color=colors[1], font_size=28),),   
        legend_opts=opts.LegendOpts(is_show=False),
    )
)
P.render_notebook()

# Friendly赛事比赛最多，几乎占据了半壁江山，其次是FIFA World Cup qualification

# 关于哪些球队热衷于参加友谊赛

In [63]:
df_friend=df1[df1['赛事名']=='Friendly']

In [64]:
from collections import Counter
hometeam=df_friend['主队队名'].tolist()
custeam=df_friend['客队队名'].tolist()
homedict=Counter(hometeam)
cusdict=Counter(custeam)
#将客队的队伍数量与主队的队伍数量相加（字典中进行键值相同的叠加）
homedict.update(cusdict)
homedict

Counter({'Scotland': 227,
         'England': 406,
         'Wales': 147,
         'Northern Ireland': 112,
         'USA': 345,
         'Uruguay': 299,
         'Austria': 451,
         'Hungary': 557,
         'Argentina': 354,
         'Belgium': 442,
         'France': 480,
         'Netherlands': 447,
         'Czechoslovakia': 281,
         'Switzerland': 452,
         'Sweden': 534,
         'Germany': 565,
         'Italy': 384,
         'Chile': 299,
         'Norway': 388,
         'Finland': 322,
         'Luxembourg': 137,
         'Russia': 341,
         'Denmark': 344,
         'Japan': 231,
         'Paraguay': 247,
         'Brazil': 414,
         'Canada': 195,
         'Estonia': 243,
         'Costa Rica': 206,
         'Guatemala': 192,
         'Spain': 329,
         'Poland': 520,
         'Yugoslavia': 258,
         'New Zealand': 206,
         'Romania': 340,
         'Latvia': 160,
         'Portugal': 271,
         'China': 278,
         'Australia': 258,
   

In [65]:
index=['总共参与场数']
dffriendly=pd.DataFrame(homedict,index=index)
dffriendly=dffriendly.T.sort_values(by='总共参与场数',ascending=False)
x_axis=dffriendly.index.tolist()[::-1]
y_axis=dffriendly['总共参与场数'].tolist()[::-1]

In [66]:
bar1 = (
    Bar()
    .add_xaxis(x_axis)
    .add_yaxis("友谊赛中总的参与场数",y_axis, color='#EC407A')
    .set_series_opts(
         label_opts=opts.LabelOpts(
            position='inside',
            vertical_align='middle',
         ),
        itemstyle_opts={
            'opacity': 0.9,
            'shadowBlur': 5,
            'shadowOffsetX': 5,
            'shadowOffsetY': 5,
            'shadowColor': 'rgba(125,0, 125, 0.5)',
            'barBorderRadius': [0, 15, 15, 0],
        }
      )
    .set_global_opts(
        tooltip_opts=opts.TooltipOpts(is_show=True, trigger='axis', axis_pointer_type='cross'),        
        datazoom_opts=opts.DataZoomOpts(orient="vertical",range_start=70,range_end=100),
        title_opts=opts.TitleOpts(
            title='友谊赛赛道的忠实参与者',
            
            pos_left='center',
            pos_top='1%',
            title_textstyle_opts=opts.TextStyleOpts(color='#ffb900', font_size=18),
        ),
        legend_opts=opts.LegendOpts(pos_left="center", pos_top='7%'),
        xaxis_opts=opts.AxisOpts(
            min_=0,
            max_=600,
            axislabel_opts=opts.LabelOpts(font_size=14, color='#c2ff00'),
            axisline_opts=opts.AxisLineOpts(
                is_show=False,
                linestyle_opts=opts.LineStyleOpts(width=2, color='#DB7093'))),
        yaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(font_size=14, color='#c2ff00'),
            axisline_opts=opts.AxisLineOpts(
                is_show=False,
                linestyle_opts=opts.LineStyleOpts(width=2, color='#DB7093')
            ),
        )
    )
    .reversal_axis()    
)

grid = Grid(init_opts=opts.InitOpts(width='1000px', height='1200px',bg_color='#0d0735'))
grid.add(bar1, opts.GridOpts(pos_top="10%",pos_left="15%", pos_right="15%"), is_control_axis_index=True)
grid.render_notebook()

# 热衷于优异赛道的前三甲为：Germany,Hungary,Sweden

# 关于地缘政治问题：从以上三幅图来分析，进球数和举办的国家数在世界第二次大战期间遭遇了最低估，遭受了重创